---
jupyter: python3
execute:
  echo: true
  eval: false
  message: false
  warning: false
---

# Transformations spatiales {#sec-chap04}



## :rocket: Préambule
### :dart: Objectifs
Dans ce chapitre, nous abordons quelques techniques de traitement d'images dans le domaine spatial uniquement. Ce chapitre est aussi disponible sous la forme d'un notebook Python sur Google Colab:

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sfoucher/TraitementImagesPythonVol1/blob/main/notebooks/04-TransformationSpatiales.ipynb){target="_blank"} 

### Librairies
Les librairies qui vont être explorées dans ce chapitre sont les suivantes:
* SciPy 

* NumPy 

* OpenCV

* RasterIO 

* Geemap

* [rioxarray](https://corteva.github.io/rioxarray/stable/index.html)

* xarray 

Dans l'environnement Google Colab, seul `rioxarray` doit être installés:

In [ ]:
%%capture
!pip install -qU rioxarray xrscipy

Vérifier les importations:

In [ ]:
#| eval: true
import numpy as np
import rioxarray as rxr
from scipy import signal
import xarray as xr
import xrscipy

### Images utilisées

Nous allons utilisez les images suivantes dans ce chapitre:

In [ ]:
!wget https://github.com/sfoucher/TraitementImagesPythonVol1/raw/refs/heads/main/data/chapitre01/subset_RGBNIR_of_S2A_MSIL2A_20240625T153941_N0510_R011_T18TYR_20240625T221903.tif -O RGBNIR_of_S2A.tif
!wget https://github.com/sfoucher/opengeos-data/raw/refs/heads/main/raster/landsat7.tif -O landsat7.tif
!wget https://github.com/sfoucher/opengeos-data/raw/refs/heads/main/images/berkeley.jpg -O berkeley.jpg

Vérifiez que vous êtes capable de les lire :

In [ ]:
#| eval: true
#| output: false

with rxr.open_rasterio('berkeley.jpg', mask_and_scale= True) as img_rgb:
    print(img_rgb)
with rxr.open_rasterio('RGBNIR_of_S2A.tif', mask_and_scale= True) as img_rgbnir:
    print(img_rgbnir)

## Analyse fréquentielle

L'analyse fréquentielle, issue du traitement du signal, permet d'avoir un autre point de vue sur les données à partir de ses composantes harmoniques. La modifications des composantes de Fourier modifie l'ensemble de l'image et permet de corriger des problèmes systématiques comme des artefacts ou du bruit de capteur.

### La transformée de Fourier

La transformée de Fourier permet de transformer une image dans un espace fréquentielle. Cette transformée est complètement reversible. Dans le cas des images numériques, on parle de `2D-FFT` (*2D-Fast Fourier Transform*). 

Dans le cas d'un signal périodique:

In [ ]:
#| eval: true
arr = xr.DataArray(np.sin(np.linspace(0, 15.7, 30)) ** 2,
                   dims=('x'), coords={'x': np.linspace(0, 5, 30)})

### Filtrage fréquentielle


### L'aliasing

## Filtrage d'image

Le filtrage d'image a plusieurs objectifs en télédétection:

1. La réduction du bruit afin d'améliorer la résolution radiométrique et améliorer la lisibilité de l'image.

2. Le réhaussement de l'image afin d'améliorer le contraste ou faire ressortir les contours.

3. La production de nouvelles caractéristiques: c.à.d dériver de nouvelles images mettant en valeur certaines informations dans l'image comme la texture, les contours, etc.

Il existe de nombreuses méthodes de filtrage dans la littérature, on peut rassembler ces filtres en quatre grandes catégories: 

1. Le filtrage peut-être global ou local, c.à.d prendre en compte toute l'image pour filtrer (ex: filtrage par Fourier) ou seulement localement avec une fenêtre ou un voisinage local.  

2. La fonction de filtrage peut-être linéaire ou non linéaire.

3. La fonction de filtrage peut être stationnaire ou adaptative

4. Le filtrage peut-être mono-échelle ou multi-échelles 

### Filtrage linéaire stationnaire

Un filtrage linéaire stationnaire consiste à appliquer une même pondération locale des valeurs des pixels dans une fenêtre glissante. La taille de cette fenêtre est généralement impaire (3,5, etc.) afin de définir une position centrale et une fenêtre symétrique.
 
:::{.callout-note}
Mettre une figure ici
:::

Le filtre le plus simple est certainement le filtre moyen qui consiste à appliquer le même poids uniforme dans la fenêtre glissante.

$$
F= \frac{1}{25}\left[
\begin{array}{c|c|c|c|c}
1 & 1 & 1 & 1 & 1 \\
\hline
1 & 1 & 1 & 1 & 1 \\
\hline
1 & 1 & 1 & 1 & 1 \\
\hline
1 & 1 & 1 & 1 & 1 \\
\hline
1 & 1 & 1 & 1 & 1
\end{array}
\right]
$$

En python, on dispose des fonctions `rolling` et `sliding_window` définis dans la librairie numpy. Par exemple pour le cas du filtre moyen on peut construire une nouvelle vue de l'image avec deux nouvelles  dimensions `x_win` et `y_win`:

In [ ]:
#| eval: true
import rioxarray as rxr
rolling_win = img_rgb.rolling(x=5, y=5,  min_periods= 3, center= True).construct(x="x_win", y="y_win", keep_attrs= True)
print(rolling_win[0,0,1,...])
print(rolling_win.shape)

L'avantage de cette approche est qu'il n'y a pas d'utilisation inutile de la mémoire. Noter les `nan` sur les bords de l'image car la fenêtre déborde sur les bordures de l'image. Par la suite un opérateur moyenne peut être appliqué.

In [ ]:
#| eval: true
filtre_moyen= rolling_win.mean(dim= ['x_win', 'y_win'], skipna= True)
#print(median_filter)
filtre_moyen.astype('int').plot.imshow(rgb="band")

:::{.callout-note}
Filtre de Sobel, filtre Prewitt
:::

#### Filtrage par convolution

La façon la plus efficace d'appliquer un filtre linéaire est d'appliquer une convolution. La convolution est généralement très efficace car elle est peut être calculée dans le domaine fréquentielle.

In [ ]:
#| eval: true

scharr = np.array([[ -3-3j, 0-10j,  +3 -3j],
                   [-10+0j, 0+ 0j, +10 +0j],
                   [ -3+3j, 0+10j,  +3 +3j]]) # Gx + j*Gy
print(img_rgb.isel(band=0).shape)
grad = signal.convolve2d(img_rgb.isel(band=0), scharr, boundary='symm', mode='same')
# on reconstruit un xarray à partir du résultat:
arr = xr.DataArray(np.abs(grad), dims=("y", "x"), coords= {'x': img_rgb.coords['x'], 'y': img_rgb.coords['y'], 'spatial_ref': 0})
print(arr)
arr.plot.imshow()

#### Filtrage par une couche convolutionnelle

Une couche convolutionnelle est simplement un ensemble de filtres appliqués sur la donnée d'entrée. Ce type de filtrage est à la base des réseaux dits convolutionnels qui seront abordés dans le tome 2.

### Filtrage adaptatif

Les filtrages adaptatifs consistent à appliquer un traitement en fonction du contenu local d'une image. Le filtre n'est alors plus stationnaire et sa réponse peut varier en fonction du contenu local.

## Segmentation {#sec-043}

## Vectorisation et rasterisation {#sec-044}

## Analyse de terrain {#sec-045}

### Élévation {#sec-0451}
### Pente {#sec-0452}
### Ombrage{#sec-0453}
### Visibilité {#sec-0454}


## Quiz de révision du chapitre {#sec-046}

In [ ]:
#| label: quizChapitre06
#| echo: false
#| eval: false
#| warning: false

source("code_complementaire/QuizzFunctions.R")
Chap06Quiz <- quizz("quiz/Chap06.yml", "Chap06")
render_quizz(Chap06Quiz)

## Exercices de révision {#sec-047}